In [ ]:
from numpy.random import randint
from random import choice
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import os
from imutils import paths

In [ ]:
def encode_to_gen(n, size_gen):
    binary = bin(n)[2:]
    gen = [0] * size_gen
    start_gen = size_gen - len(binary)
    gen[start_gen:] = [int(x) for x in binary]
    return gen

def decode_gen(gen):
    return int('0b' + ''.join(str(i) for i in gen), 2)

In [ ]:
class Individual:
    def __init__(self):
        self.epoch = randint(0, 4)                       # epoch
        self.input_size = randint(0, 4)                  # input size
        self.batch_size = randint(0, 4)                  # batch size
        # conv 1st
        self.filter_size_1 = randint(0, 8)               # filter size 1
        self.kernel_size_1 = randint(0, 2)               # kernel size 1
        self.pooling_1 = randint(0, 2)                   # type pooling 1
        self.pooling_size_1 = randint(0, 2)              # pooling size 1
        self.activation_1 = randint(0, 2)                # type activation 1
        self.padding_1 = randint(0, 2)                   # type padding conv 1
        # conv 2nd
        self.filter_size_2 = randint(0, 8)               # filter size 2
        self.kernel_size_2 = randint(0, 2)               # kernel size 2
        self.pooling_2 = randint(0, 2)                   # type pooling 2
        self.pooling_size_2 = randint(0, 2)              # pooling size 2
        self.activation_2 = randint(0, 2)                # type activation 2
        self.padding_2 = randint(0, 2)                   # type padding conv 2
        # conv 3th
        self.filter_size_3 = randint(0, 8)               # filter size 3
        self.kernel_size_3 = randint(0, 2)               # kernel size 3
        self.pooling_3 = randint(0, 2)                   # type pooling 3
        self.pooling_size_3 = randint(0, 2)              # pooling size 3
        self.activation_3 = randint(0, 2)                # type activation 3
        self.padding_3 = randint(0, 2)                   # type padding conv 3

        self.activation_4 = randint(0, 2)                # type activation 4
        self.activation_5 = 'softmax'                    # type activation 5
        self.dropout_1 = randint(0, 4)                   # dropout size 1
        self.dense_1 = randint(0, 8)                     # dense size 1
        self.loss_func = randint(0, 2)                   # loss function
        self.optimizer = randint(0, 2)                   # optimization type
        self.learn_rate = randint(0, 4)                  # learn rate

    # Get binary gen
    def individual_binary(self):
        # init individual with 40 parameter
        indiv = np.array([0]*40)

        # ______6 bit______|_____________8 bit x 3 = 24 bit____________|_________________10 bit__________________
        # epoch|input|batch|filter|kernel|activ|pool |pool_size|padding|dropout|dense|activ|loss_f|optim|learn_r| 
        # 2 bit|2 bit|2 bit|3 bit |1 bit |1 bit|1 bit|  1 bit  | 1 bit | 2 bit |3 bit|1 bit|1 bit |1 bit| 2 bit |

        bit = [2, 2, 2] + [3, 1, 1, 1, 1, 1] * 3 + [2, 3, 1, 1, 1, 2]
        param = [self.epoch, self.input_size, self.batch_size, self.filter_size_1, self.kernel_size_1,
                 self.pooling_1, self.pooling_size_1, self.activation_1, self.padding_1, self.filter_size_2, 
                 self.kernel_size_2, self.pooling_2, self.pooling_size_2, self.activation_2, self.padding_2, 
                 self.filter_size_3, self.kernel_size_3, self.pooling_3, self.pooling_size_3, self.activation_3, 
                 self.padding_3, self.dropout_1, self.dense_1, self.activation_4, self.loss_func, self.optimizer, 
                 self.learn_rate]
        start = 0
        for i in range(len(param)):
            indiv[start:start+bit[i]] = encode_to_gen(param[i], bit[i])
            start += bit[i]
        
        return indiv

In [ ]:
    # Get model 
    def individual_model(indiv, num_class): 
        input_shape=(indiv['input'], indiv['input'], 3)
        model = Sequential()
        # conv 1st
        model.add(layer=Conv2D(indiv['filter_1'], indiv['kernel_1'], padding=indiv['padding_1'], activation=indiv['activation_1'], input_shape=input_shape))
        if indiv['pooling_1'] == 'max':
            model.add(layer=MaxPooling2D(indiv['pooling_size_1'][0]))
        elif indiv['pooling_1'] == 'average':
            model.add(layer=AveragePooling2D(indiv['pooling_size_1'][0]))
        # conv 2nd
        model.add(layer=Conv2D(indiv['filter_2'], indiv['kernel_2'], padding=indiv['padding_2'], activation=indiv['activation_2']))
        if indiv['pooling_2'] == 'max':
            model.add(layer=MaxPooling2D(indiv['pooling_size_2'][0]))
        elif indiv['pooling_2'] == 'average':
            model.add(layer=AveragePooling2D(indiv['pooling_size_2'][0]))
        # conv 3th
        model.add(layer=Conv2D(indiv['filter_3'], indiv['kernel_3'], padding=indiv['padding_3'], activation=indiv['activation_3']))
        if indiv['pooling_3'] == 'max':
            model.add(layer=MaxPooling2D(indiv['pooling_size_3'][0]))
        elif indiv['pooling_3'] == 'average':
            model.add(layer=AveragePooling2D(indiv['pooling_size_3'][0]))
        # dense layer
        model.add(layer=Flatten())
        model.add(layer=Dropout(indiv['dropout_1']))
        model.add(layer=Dense(indiv['dense_1'], activation=indiv['activation_4']))
        model.add(layer=Dense(num_class, activation=indiv['activation_5']))

        # plot_model(model, to_file='images/model_plot_{}.png'.format(index_p), show_shapes=True, show_layer_names=True)
        # choose type to optimizer
        if indiv['optimizer'] == 'adam':
            opt = Adam(lr=indiv['learn_rate'], decay=indiv['learn_rate']/ indiv['epoch'])
        else:
            opt = Adamax(lr=indiv['learn_rate'], decay=indiv['learn_rate']/ indiv['epoch'])
        model.compile(optimizer=opt, loss=indiv['loss_func'], metrics=['accuracy'])
        
        # construct the training image generator for data augmentation
        aug = ImageDataGenerator(
            rotation_range=40,
            zoom_range=0.15,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            fill_mode="nearest")
        # train the head of the network

        x_train, x_test, y_train, y_test = mask_detect(input_shape[:2])

        print("[INFO] training head...")
        H = model.fit(
            aug.flow(x_train, y_train, batch_size=indiv['batch']),
            steps_per_epoch=len(x_train) // indiv['batch'],
            validation_data=(x_test, y_test),
            validation_steps=len(x_test) // indiv['batch'],
            epochs=indiv['epoch'])

        fitness = model.evaluate(x=x_test, y=y_test, verbose=0)[1]
        return fitness

In [ ]:
    def individual_decode(binary_gen):
        input_size = [70, 120, 170, 220]
        batch = [10, 15, 20, 25]
        kernel_size = [(3, 3), (5, 5)]
        pooling = ['max', 'average']
        pooling_size = [(2, 2), (3, 3)]
        activation = ['sigmoid', 'relu']
        padding = ['valid', 'same']
        dropout = [0.1, 0.2, 0.3, 0.4]
        loss_func = ['categorical_crossentropy', 'binary_crossentropy']
        learn_rate = [1e-2, 1e-3, 1e-4, 1e-5]
        optimizer = ['adamax', 'adam']

        params = {'epoch': decode_gen(binary_gen[:2]) + 5,                             
                'input': input_size[decode_gen(binary_gen[2:4])],                 
                'batch': batch[decode_gen(binary_gen[4:6])],                      
                # conv 1st (8 bit)
                'filter_1': decode_gen(binary_gen[6:9]) + 30,                 
                'kernel_1': kernel_size[decode_gen(binary_gen[9:10])],          
                'pooling_1': pooling[decode_gen(binary_gen[10:11])],                 
                'pooling_size_1': pooling_size[decode_gen(binary_gen[11:12])],  
                'activation_1': activation[decode_gen(binary_gen[12:13])],        
                'padding_1': padding[decode_gen(binary_gen[13:14])],               
                # conv 2nd (8 bit)
                'filter_2': decode_gen(binary_gen[14:17]) + 60,
                'kernel_2': kernel_size[decode_gen(binary_gen[17:18])],
                'pooling_2': pooling[decode_gen(binary_gen[18:19])],
                'pooling_size_2': pooling_size[decode_gen(binary_gen[19:20])],
                'activation_2': activation[decode_gen(binary_gen[20:21])],
                'padding_2': padding[decode_gen(binary_gen[21:22])],
                # conv 3th (8 bit)
                'filter_3': decode_gen(binary_gen[22:25]) + 120,
                'kernel_3': kernel_size[decode_gen(binary_gen[25:26])],
                'pooling_3': pooling[decode_gen(binary_gen[26:27])],
                'pooling_size_3': pooling_size[decode_gen(binary_gen[27:28])],
                'activation_3': activation[decode_gen(binary_gen[28:29])],
                'padding_3': padding[decode_gen(binary_gen[29:30])],
                # dense 1
                # 2 bit
                'dropout_1': dropout[decode_gen(binary_gen[30:32])],
                # 3 bit
                'dense_1': decode_gen(binary_gen[32:35]) + 250,
                # 1 bit
                'activation_4': activation[decode_gen(binary_gen[35:36])],
                'activation_5': 'softmax',
                # loss function, optimizer & learn rate
                # 1 bit
                'loss_func': loss_func[decode_gen(binary_gen[36:37])],
                # 1 bit
                'optimizer': optimizer[decode_gen(binary_gen[37:38])],
                # 2 bit
                'learn_rate': learn_rate[decode_gen(binary_gen[38:40])] 
                }
        return params

In [ ]:
class Population():
    def __init__(self, num_individual = 10, score_func = None, 
    prob_mut = 0.2, prob_crossover = .5, num_generation = 100, 
    crossover_method = 'single', verbose = True, threshold = 1):

        self.score_func = score_func
        self.n_population = num_individual
        self.prob_mut = prob_mut
        self.prob_crossover = prob_crossover
        self.population = [indiv.individual_binary() for indiv in [Individual() for _ in range(num_individual)]]
        self.scores = []
        self.num_generation = num_generation
        self.best_result = None
        self.crossover_ = crossover_method
        self.verbose = verbose
        self.threshold = threshold

    def eval_score(self):
        sc = np.array([self.score_func(i) for i in self.population])
        sc[sc <= 0] = 0.0000000001
        return sc

    def verbos_func(self, best_score, best_pop):
        print('Num population:', self.n_population)
        print('Current best score:',best_score)
        print('Current best individual:', best_pop)
        print(Individual().individual_decode(best_pop))
        print("_" * 50)

    def crossover(self, one, two, method = 'single'):
        if method == 'single':
            pnt = np.random.randint(len(one))
            one_temp = np.concatenate([one[0:pnt],two[pnt:]])
            two_temp = np.concatenate([two[0:pnt], one[pnt:]])
            return one_temp, two_temp

        elif method == 'multi':
            pnt1, pnt2 = np.random.randint(len(one), size=2)
            if pnt1 > pnt2:
                pnt1, pnt2 = pnt2, pnt1
            one_temp = np.concatenate([one[0:pnt1], two[pnt1: pnt2], one[pnt2:]])
            two_temp = np.concatenate([two[0:pnt1], one[pnt1: pnt2], two[pnt2:]])
            return one_temp, two_temp
        elif method == 'uniform':
            pr = np.random.rand(len(one)) > 0.
            one_temp = one
            two_temp = two
            for i in range(len(one)):
                if pr[i]:
                    one_temp[i], two_temp[i] = two_temp[i], one_temp[i]
            return  one_temp, two_temp

    def mutation(self, one):
        temp_rand = np.random.randint(len(one))
        one[temp_rand] = 1 - one[temp_rand]
        return one

    def choose(self, scores):
        indexs = np.random.choice(np.arange(len(scores)), size = 2, p = scores/scores.sum())
        return indexs

    def inititalization(self):
        self.population = np.array([indiv.individual_binary() for indiv in [Individual() for _ in range(self.n_population)]])
        self.scores = self.eval_score()

    def print_pop(self):
        print('last population ', self.population)
        print('best result ', self.best_result)

    def fit(self):
        self.inititalization()
        best_idx = np.argsort(self.scores)[-1]
        best_score = self.scores[best_idx]
        best_pop = self.population[best_idx, :]

        for g in range(self.num_generation):
            if best_score >= self.threshold:
                self.verbos_func(best_score, best_pop)
                break
            print("Generation {}:\n".format(g+1))
            print("Individuals & Fitness: \n{}".format([i for i in zip(self.population, self.scores)]))
            new_generation = []
            for n in range(self.n_population // 2):
                # print('dfsdf', type(self.scores))
                i, j = self.choose(self.scores)

                one = self.population[i, :].copy()
                two = self.population[j, :].copy()

                if np.random.rand() < self.prob_crossover:

                    one ,two = self.crossover(one, two, self.crossover_)

                if np.random.rand() < self.prob_mut:
                    one = self.mutation(one)
                    two = self.mutation(two)

                new_generation.append(one)
                new_generation.append(two)

            self.population = np.array(new_generation)
            self.scores = self.eval_score()

            if np.max(self.scores) > best_score:
                best_idx = np.argsort(self.scores)[-1]
                best_score = self.scores[best_idx]
                best_pop = self.population[best_idx, :]
            else:
                worst_idx = np.argsort(self.scores)[0]
                self.population[worst_idx, :] = best_pop
                self.scores[worst_idx] = best_score

            if self.verbose:
                self.verbos_func(best_score, best_pop)
        self.best_result = best_pop

In [ ]:
def mask_detect(img_shape):
    # grab the list of images in our dataset directory, then initialize
    # the list of data (i.e., images) and class images
    print("[INFO] loading images...")
    imagePaths = list(paths.list_images("data"))
    data = []
    labels = []
    # loop over the image paths
    for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]

        # load the input image and preprocess it
        image = load_img(imagePath, target_size=img_shape)
        image = img_to_array(image)
        image = preprocess_input(image)

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    # convert the data and labels to NumPy arrays
    data = np.array(data, dtype="float32")
    labels = np.array(labels)

    # perform one-hot encoding on the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = to_categorical(labels)
    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    (x_train, x_test, y_train, y_test) = train_test_split(data, labels,
        test_size=0.20, stratify=labels, random_state=42)
    
    print("[INFO] done load...")
    return x_train, x_test, y_train, y_test

In [ ]:
def fitness(indiv_gen): 
    # Init fitness
    indiv = individual_decode(indiv_gen)

    fitness_indi = individual_model(indiv=indiv, num_class=2)

    # print('Accuracy: {}'.format(fitness_indi * 100))
    return fitness_indi

In [ ]:
n_individual = 2     # Number of individual
n_generation = 1     # Number of generation
threshold = 0.994    # Threshold

model = Population(num_generation=n_generation, 
                    num_individual=n_individual, 
                    score_func=fitness, 
                    prob_crossover=.8, 
                    prob_mut=.3, 
                    crossover_method='uniform',
                    threshold=threshold)
model.fit()